### Multi-Class Classification

### Random Forest

In [1]:
#Import libraries to open data file
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.svm import SVC
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, balanced_accuracy_score
from sklearn.model_selection import RandomizedSearchCV
import pprint
from sklearn.metrics import confusion_matrix

In [2]:
#Read in NTA Normalized dataset(92 samples x 593 features). This is post converting log10 normalization. All 0s are 1/10th lowest conc of the entire dataset (Stults cite)
dat_nt = pd.read_csv(r'240603-NTA-Normalized-MultiClass-Input.csv', header=0)
#print(dat_nt.head)
#print(dat_nt.shape)

In [3]:
#Convert to numpy
dat_nt_np = dat_nt.to_numpy()
target_1 = dat_nt_np[:,0] #Convert target variables to 2D-array for sci-kit learn
data_1 = dat_nt_np[:,1:]
#print(target_1.shape)
#print(data_1.shape)

In [4]:
#Running randomized CV to narrow grid for GridSearch CV
#Modified from here: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 2000, num = 200)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790, 800, 810, 820, 830, 840, 850, 860, 870, 880, 890, 900, 910, 920, 930, 940, 950, 960, 970, 980, 990, 1000, 1010, 1020, 1030, 1040, 1050, 1060, 1070, 1080, 1090, 1100, 1110, 1120, 1130, 1140, 1150, 1160, 1170, 1180, 1190, 1200, 1210, 1220, 1230, 1240, 1250, 1260, 1270, 1280, 1290, 1300, 1310, 1320, 1330, 1340, 1350, 1360, 1370, 1380, 1390, 1400, 1410, 1420, 1430, 1440, 1450, 1460, 1470, 1480, 1490, 1500, 1510, 1520, 1530, 1540, 1550, 1560, 1570, 1580, 1590, 1600, 1610, 1620, 1630, 1640, 1650, 1660, 1670, 1680, 1690, 1700, 1710, 1720, 1730, 1740, 1750, 1760, 1770, 1780, 1790, 1800, 1810, 1820

In [7]:
# Split the data into training and held-out validation set
data_train, data_val, target_train, target_val = train_test_split(data_1, target_1, test_size=0.2, random_state=42, stratify=target_1)

# Use the random grid to search for best hyperparameters
# First create the base model to tune
model_rnd = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=model_rnd, param_distributions=random_grid, n_iter=100, cv=6, verbose=2, random_state=42, n_jobs=-1)

# Fit the random search model on the training data
rf_random.fit(data_train, target_train)

# Print the best parameters found
print("Best Parameters:", rf_random.best_params_)

Fitting 6 folds for each of 100 candidates, totalling 600 fits
Fitting 6 folds for each of 100 candidates, totalling 600 fits
Best Parameters: {'n_estimators': 1070, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': None, 'bootstrap': False}
Best Parameters: {'n_estimators': 1070, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': None, 'bootstrap': False}


In [8]:
# Evaluate the best model on the held-out validation set
best_model = rf_random.best_estimator_
val_predictions = best_model.predict(data_val)
val_balanced_accuracy = balanced_accuracy_score(target_val, val_predictions)

print("Validation Balanced Accuracy:", val_balanced_accuracy)

Validation Balanced Accuracy: 0.8194444444444443
Validation Balanced Accuracy: 0.8194444444444443


In [ ]:
# Split the data into training and held-out validation set
data_train, data_val, target_train, target_val = train_test_split(data_1, target_1, test_size=0.2, random_state=42, stratify=target_1)

# Define the model
model = RandomForestClassifier()

# Parameters to explore
n_estimators = [1060, 1070]
max_features = ['log2']
max_depth = [100, None]
min_samples_leaf = [1, 2]
min_samples_split = [5, 6]
bootstrap = [False]

# Create a custom scorer for balanced accuracy
balanced_accuracy_scorer = make_scorer(balanced_accuracy_score)

# Define the grid
grid = dict(n_estimators=n_estimators, max_features=max_features, max_depth=max_depth,
            min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, bootstrap=bootstrap)

# Perform grid search with cross-validation
cv = RepeatedStratifiedKFold(n_splits=6, n_repeats=10, random_state=42)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring=balanced_accuracy_scorer, error_score=0)
grid_result = grid_search.fit(data_train, target_train.ravel())

# Summarize results of training data
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Evaluate the best model on the held-out validation set
best_model = grid_result.best_estimator_
val_predictions = best_model.predict(data_val)
val_balanced_accuracy = balanced_accuracy_score(target_val, val_predictions)

print("Validation Balanced Accuracy:", val_balanced_accuracy)

In [ ]:
#Now running optimized model over the entire dataset
model = RandomForestClassifier(n_estimators = 1060, max_features = 'log2', max_depth = 100, min_samples_leaf = 1, min_samples_split = 5, bootstrap = False)
# fit the model
model.fit(data_1, target_1.ravel())
acc = model.score(data_1, target_1)

print("RF accuracy: " + str(round(acc, 3)))

In [ ]:
# Predict class labels for the data
predicted_labels = model.predict(data_1)

# Initialize lists to store data points, row numbers, and original target labels for each class
class_data = [[] for _ in range(6)]  # Assuming 6 classes
class_row_numbers = [[] for _ in range(6)]
class_original_labels = [[] for _ in range(6)]

# Organize samples into their respective classes
for row_number, (predicted_label, original_label) in enumerate(zip(predicted_labels, target_1)):
    class_data[int(predicted_label)].append(data_1[row_number])
    class_row_numbers[int(predicted_label)].append(row_number)
    class_original_labels[int(predicted_label)].append(original_label)

# Print information for each class
for class_idx in range(6):
    print(f"Class {class_idx}:")
    print(f"Number of data points: {len(class_data[class_idx])}")
    print(f"Row numbers: {class_row_numbers[class_idx]}")
    print(f"Original target labels: {class_original_labels[class_idx]}")
    print()

In [ ]:
# Mapping from class numbers to class types
class_mapping = {
    0.0: "AFFF-GW",
    1.0: "LL",
    2.0: "BL",
    3.0: "WWTP",
    4.0: "PP",
    5.0: "PG"
}

# Replace class numbers with class types in class_original_labels
class_original_labels = [
    [class_mapping.get(label, label) for label in class_labels]
    for class_labels in class_original_labels
]
print(class_original_labels)

In [ ]:
# Define class types in the same order as the mapping
class_types = ["AFFF-GW", "LL", "BL", "WWTP", "PP", "PG"]

# Initialize six NumPy arrays to count instances for each class type
y1 = np.zeros(len(class_original_labels))
y2 = np.zeros(len(class_original_labels))
y3 = np.zeros(len(class_original_labels))
y4 = np.zeros(len(class_original_labels))
y5 = np.zeros(len(class_original_labels))
y6 = np.zeros(len(class_original_labels))

# Iterate through class_original_labels to count instances for each class type
for i, labels in enumerate(class_original_labels):
    for j, class_type in enumerate(class_types):
        # Count the instances of the current class type in the current class
        count = labels.count(class_type)
        
        # Update the respective NumPy array (e.g., y1, y2, etc.)
        if j == 0:
            y1[i] = count
        elif j == 1:
            y2[i] = count
        elif j == 2:
            y3[i] = count
        elif j == 3:
            y4[i] = count
        elif j == 4:
            y5[i] = count
        elif j == 5:
            y6[i] = count
            
print(y1)
print(y2)
print(y3)
print(y4)
print(y5)
print(y6)

In [ ]:
#create bar plot
x = ['AFFF-GW', 'LL', 'BL', 'WWTP', 'PP', 'PG']

plt.figure(figsize=(10, 6)) 
plt.bar(x, y1, color ='#7ad151')
plt.bar(x, y2, bottom=y1, color='#22a884')
plt.bar(x, y3, bottom=y1+y2, color='#fde725')
plt.bar(x, y4, bottom=y1+y2+y3, color='#440154')
plt.bar(x, y5, bottom=y1+y2+y3+y4, color='#414487')
plt.bar(x, y6, bottom=y1+y2+y3+y4+y5, color='#2a788e')
plt.yticks(np.arange(0, 29, 2), fontsize = 12)
plt.xticks(fontsize = 12)
plt.xlabel("RF Groups", fontsize = 14)
plt.ylabel("Frequency", fontsize = 14)
for i, (yi, yj, yk, yl, ym, yn) in enumerate(zip(y1, y2, y3, y4, y5, y6)):
    total_count = int(yi + yj + yk + yl + ym + yn)  # Convert to int to remove decimals
    plt.text(i, total_count + 0.5, str(total_count), ha="center")
#plt.legend(["AFFF-GW",  "LL", "BL", "WWTP", "PP", "PG"], loc="upper right")
#plt.title("Class breakdown via RF vs. Original Class Label", fontsize = 16)
plt.savefig('RF-MultiClass-RNKU-Normalized_NEW.png', dpi = 1500, bbox_inches='tight')
plt.show()

In [ ]:
#create bar plot
x = ['AFFF-GW', 'LL', 'BL', 'WWTP', 'PP', 'PG']

plt.figure(figsize=(10, 6)) 
plt.bar(x, y1, color ='#7ad151')
plt.bar(x, y2, bottom=y1, color='#22a884')
plt.bar(x, y3, bottom=y1+y2, color='#fde725')
plt.bar(x, y4, bottom=y1+y2+y3, color='#440154')
plt.bar(x, y5, bottom=y1+y2+y3+y4, color='#414487')
plt.bar(x, y6, bottom=y1+y2+y3+y4+y5, color='#2a788e')
plt.yticks(np.arange(0, 29, 2), fontsize = 12)
plt.xticks(fontsize = 12)
plt.xlabel("RF Groups", fontsize = 14)
plt.ylabel("Frequency", fontsize = 14)
for i, (yi, yj, yk, yl, ym, yn) in enumerate(zip(y1, y2, y3, y4, y5, y6)):
    total_count = int(yi + yj + yk + yl + ym + yn)  # Convert to int to remove decimals
    plt.text(i, total_count + 0.5, str(total_count), ha="center")
plt.legend(["AFFF-GW",  "LL", "BL", "WWTP", "PP", "PG"], loc="upper right")
#plt.title("Class breakdown via RF vs. Original Class Label", fontsize = 16)
plt.savefig('RF-MultiClass-RNKU-Normalized-Legend_NEW.png', dpi = 1500, bbox_inches='tight')
plt.show()

### Logistic Regression

In [ ]:
#Running randomized CV to narrow grid for GridSearch CV
#Modified from here: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
solver = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l1','l2']
C = np.logspace(-2, 10, 13)

# Create the random grid
random_grid = {'solver': solver,
               'penalty': penalty,
               'C': C.tolist()}
print(random_grid)

In [ ]:
# Split the data into training and held-out validation set
data_train, data_val, target_train, target_val = train_test_split(data_1, target_1, test_size=0.2, random_state=42)

# Use the random grid to search for best hyperparameters
# First create the base model to tune
model_rnd = LogisticRegression()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
lr_random = RandomizedSearchCV(estimator=model_rnd, param_distributions=random_grid, n_iter=100, cv=7, verbose=2, random_state=42, n_jobs=-1)

# Fit the random search model on the training data
lr_random.fit(data_train, target_train)

# Print the best parameters found
print("Best Parameters:", lr_random.best_params_)

In [ ]:
# Evaluate the best model on the held-out validation set
best_model = lr_random.best_estimator_
val_predictions = best_model.predict(data_val)
val_balanced_accuracy = balanced_accuracy_score(target_val, val_predictions)

print("Validation Balanced Accuracy:", val_balanced_accuracy)

In [ ]:
# Split the data into training and held-out validation set
data_train, data_val, target_train, target_val = train_test_split(data_1, target_1, test_size=0.2, random_state=42)

#Defining model based on results from randomized CV
model = LogisticRegression()

#Parameters to explore
solver = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
C = [0.1, 10, 100]

#Create a custom scorer for balanced accuracy
balanced_accuracy_scorer = make_scorer(balanced_accuracy_score)

#Defining grid and performing CV of training data
grid = dict(solver=solver, penalty=penalty, C=C)
cv = RepeatedStratifiedKFold(n_splits=7, n_repeats=10, random_state=42)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring=balanced_accuracy_scorer, error_score=0)
grid_result = grid_search.fit(data_train, target_train.ravel())

# Summarize results of training data
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Evaluate the best model on the held-out validation set
best_model = grid_result.best_estimator_
val_predictions = best_model.predict(data_val)
val_balanced_accuracy = balanced_accuracy_score(target_val, val_predictions)

print("Validation Balanced Accuracy:", val_balanced_accuracy)



In [ ]:
#Now running optimized model over the entire dataset
model = LogisticRegression(C=10, penalty = 'l2', solver='newton-cg')
# fit the model
model.fit(data_1, target_1.ravel())
acc = model.score(data_1, target_1)

print("LR accuracy: " + str(round(acc, 3)))

In [ ]:
# Predict class labels for the data
predicted_labels = model.predict(data_1)

# Initialize lists to store data points, row numbers, and original target labels for each class
class_data = [[] for _ in range(6)]  # Assuming 6 classes
class_row_numbers = [[] for _ in range(6)]
class_original_labels = [[] for _ in range(6)]

# Organize samples into their respective classes
for row_number, (predicted_label, original_label) in enumerate(zip(predicted_labels, target_1)):
    class_data[int(predicted_label)].append(data_1[row_number])
    class_row_numbers[int(predicted_label)].append(row_number)
    class_original_labels[int(predicted_label)].append(original_label)

# Print information for each class
for class_idx in range(6):
    print(f"Class {class_idx}:")
    print(f"Number of data points: {len(class_data[class_idx])}")
    print(f"Row numbers: {class_row_numbers[class_idx]}")
    print(f"Original target labels: {class_original_labels[class_idx]}")
    print()

In [ ]:
# Mapping from class numbers to class types
class_mapping = {
    0.0: "AFFF-GW",
    1.0: "LL",
    2.0: "BL",
    3.0: "WWTP",
    4.0: "PP",
    5.0: "PG"
}

# Replace class numbers with class types in class_original_labels
class_original_labels = [
    [class_mapping.get(label, label) for label in class_labels]
    for class_labels in class_original_labels
]
print(class_original_labels)

In [ ]:
# Define class types in the same order as the mapping
class_types = ["AFFF-GW", "LL", "BL", "WWTP", "PP", "PG"]

# Initialize six NumPy arrays to count instances for each class type
y1 = np.zeros(len(class_original_labels))
y2 = np.zeros(len(class_original_labels))
y3 = np.zeros(len(class_original_labels))
y4 = np.zeros(len(class_original_labels))
y5 = np.zeros(len(class_original_labels))
y6 = np.zeros(len(class_original_labels))

# Iterate through class_original_labels to count instances for each class type
for i, labels in enumerate(class_original_labels):
    for j, class_type in enumerate(class_types):
        # Count the instances of the current class type in the current class
        count = labels.count(class_type)
        
        # Update the respective NumPy array (e.g., y1, y2, etc.)
        if j == 0:
            y1[i] = count
        elif j == 1:
            y2[i] = count
        elif j == 2:
            y3[i] = count
        elif j == 3:
            y4[i] = count
        elif j == 4:
            y5[i] = count
        elif j == 5:
            y6[i] = count
            
print(y1)
print(y2)
print(y3)
print(y4)
print(y5)
print(y6)

In [ ]:
#create bar plot
x = ['AFFF-GW', 'LL', 'BL', 'WWTP', 'PP', 'PG']

plt.figure(figsize=(10, 6)) 
plt.bar(x, y1, color ='#7ad151')
plt.bar(x, y2, bottom=y1, color='#22a884')
plt.bar(x, y3, bottom=y1+y2, color='#fde725')
plt.bar(x, y4, bottom=y1+y2+y3, color='#440154')
plt.bar(x, y5, bottom=y1+y2+y3+y4, color='#414487')
plt.bar(x, y6, bottom=y1+y2+y3+y4+y5, color='#2a788e')
plt.yticks(np.arange(0, 29, 2), fontsize = 12)
plt.xticks(fontsize = 12)
plt.xlabel("LR Groups", fontsize = 14)
plt.ylabel("Frequency", fontsize = 14)
for i, (yi, yj, yk, yl, ym, yn) in enumerate(zip(y1, y2, y3, y4, y5, y6)):
    total_count = int(yi + yj + yk + yl + ym + yn)  # Convert to int to remove decimals
    plt.text(i, total_count + 0.5, str(total_count), ha="center")
#plt.legend(["AFFF-GW",  "LL", "BL", "WWTP", "PP", "PG"], loc="upper right")
#plt.title("Class breakdown via RF vs. Original Class Label", fontsize = 16)
plt.savefig('LR-MultiClass-RNKU-Normalized_NEW.png', dpi = 1500, bbox_inches='tight')
plt.show()

In [ ]:
#create bar plot
x = ['AFFF-GW', 'LL', 'BL', 'WWTP', 'PP', 'PG']

plt.figure(figsize=(10, 6)) 
plt.bar(x, y1, color ='#7ad151')
plt.bar(x, y2, bottom=y1, color='#22a884')
plt.bar(x, y3, bottom=y1+y2, color='#fde725')
plt.bar(x, y4, bottom=y1+y2+y3, color='#440154')
plt.bar(x, y5, bottom=y1+y2+y3+y4, color='#414487')
plt.bar(x, y6, bottom=y1+y2+y3+y4+y5, color='#2a788e')
plt.yticks(np.arange(0, 29, 2), fontsize = 12)
plt.xticks(fontsize = 12)
plt.xlabel("LR Groups", fontsize = 14)
plt.ylabel("Frequency", fontsize = 14)
for i, (yi, yj, yk, yl, ym, yn) in enumerate(zip(y1, y2, y3, y4, y5, y6)):
    total_count = int(yi + yj + yk + yl + ym + yn)  # Convert to int to remove decimals
    plt.text(i, total_count + 0.5, str(total_count), ha="center")
plt.legend(["AFFF-GW",  "LL", "BL", "WWTP", "PP", "PG"], loc="upper right")
#plt.title("Class breakdown via RF vs. Original Class Label", fontsize = 16)
plt.savefig('LR-MultiClass-RNKU-Normalized-Legend_NEW.png', dpi = 1500, bbox_inches='tight')
plt.show()

### Support Vector Classification

In [ ]:
#Running randomized CV to narrow grid for GridSearch CV
#Modified from here: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
C_range = np.logspace(-5, 10, 16)

# Create the random grid
random_grid = {'C': C_range.tolist()}
print(random_grid)

In [ ]:
# Split the data into training and held-out validation set
data_train, data_val, target_train, target_val = train_test_split(data_1, target_1, test_size=0.2, random_state=42)

# Use the random grid to search for best hyperparameters
# First create the base model to tune
model_rnd = LogisticRegression()
# Use the random grid to search for best hyperparameters
# First create the base model to tune
model_rnd = SVC(kernel = 'linear', tol = 0.00001, shrinking = True, cache_size = 200, verbose = False, max_iter = -1)

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
sv_random = RandomizedSearchCV(estimator = model_rnd, param_distributions = random_grid, n_iter = 16, cv = 7, verbose=2, random_state=42, n_jobs = -1, error_score = 0)
# Fit the random search model on the training data
sv_random.fit(data_train, target_train)

# Print the best parameters found
print("Best Parameters:", sv_random.best_params_)


In [ ]:
# Evaluate the best model on the held-out validation set
sv_model = sv_random.best_estimator_
val_predictions = sv_model.predict(data_val)
val_balanced_accuracy = balanced_accuracy_score(target_val, val_predictions)

print("Validation Balanced Accuracy:", val_balanced_accuracy)

In [ ]:
#Now running optimized model over the entire dataset
model = SVC(kernel = 'linear', C = 0.01, tol = 0.00001, shrinking = True, cache_size = 200, verbose = False, max_iter = -1)
# fit the model
model.fit(data_1, target_1.ravel())
acc = model.score(data_1, target_1)

print("SVC accuracy: " + str(round(acc, 3)))

In [ ]:
# Predict class labels for the data
predicted_labels = model.predict(data_1)

# Initialize lists to store data points, row numbers, and original target labels for each class
class_data = [[] for _ in range(6)]  # Assuming 6 classes
class_row_numbers = [[] for _ in range(6)]
class_original_labels = [[] for _ in range(6)]

# Organize samples into their respective classes
for row_number, (predicted_label, original_label) in enumerate(zip(predicted_labels, target_1)):
    class_data[int(predicted_label)].append(data_1[row_number])
    class_row_numbers[int(predicted_label)].append(row_number)
    class_original_labels[int(predicted_label)].append(original_label)

# Print information for each class
for class_idx in range(6):
    print(f"Class {class_idx}:")
    print(f"Number of data points: {len(class_data[class_idx])}")
    print(f"Row numbers: {class_row_numbers[class_idx]}")
    print(f"Original target labels: {class_original_labels[class_idx]}")
    print()

In [ ]:
# Mapping from class numbers to class types
class_mapping = {
    0.0: "AFFF-GW",
    1.0: "LL",
    2.0: "BL",
    3.0: "WWTP",
    4.0: "PP",
    5.0: "PG"
}

# Replace class numbers with class types in class_original_labels
class_original_labels = [
    [class_mapping.get(label, label) for label in class_labels]
    for class_labels in class_original_labels
]
print(class_original_labels)

In [ ]:
# Define class types in the same order as the mapping
class_types = ["AFFF-GW", "LL", "BL", "WWTP", "PP", "PG"]

# Initialize six NumPy arrays to count instances for each class type
y1 = np.zeros(len(class_original_labels))
y2 = np.zeros(len(class_original_labels))
y3 = np.zeros(len(class_original_labels))
y4 = np.zeros(len(class_original_labels))
y5 = np.zeros(len(class_original_labels))
y6 = np.zeros(len(class_original_labels))

# Iterate through class_original_labels to count instances for each class type
for i, labels in enumerate(class_original_labels):
    for j, class_type in enumerate(class_types):
        # Count the instances of the current class type in the current class
        count = labels.count(class_type)
        
        # Update the respective NumPy array (e.g., y1, y2, etc.)
        if j == 0:
            y1[i] = count
        elif j == 1:
            y2[i] = count
        elif j == 2:
            y3[i] = count
        elif j == 3:
            y4[i] = count
        elif j == 4:
            y5[i] = count
        elif j == 5:
            y6[i] = count
            
print(y1)
print(y2)
print(y3)
print(y4)
print(y5)
print(y6)

In [ ]:
#create bar plot
x = ['AFFF-GW', 'LL', 'BL', 'WWTP', 'PP', 'PG']

plt.figure(figsize=(10, 6)) 
plt.bar(x, y1, color ='#7ad151')
plt.bar(x, y2, bottom=y1, color='#22a884')
plt.bar(x, y3, bottom=y1+y2, color='#fde725')
plt.bar(x, y4, bottom=y1+y2+y3, color='#440154')
plt.bar(x, y5, bottom=y1+y2+y3+y4, color='#414487')
plt.bar(x, y6, bottom=y1+y2+y3+y4+y5, color='#2a788e')
plt.yticks(np.arange(0, 29, 2), fontsize = 12)
plt.xticks(fontsize = 12)
plt.xlabel("SVC Groups", fontsize = 14)
plt.ylabel("Frequency", fontsize = 14)
for i, (yi, yj, yk, yl, ym, yn) in enumerate(zip(y1, y2, y3, y4, y5, y6)):
    total_count = int(yi + yj + yk + yl + ym + yn)  # Convert to int to remove decimals
    plt.text(i, total_count + 0.5, str(total_count), ha="center")
#plt.legend(["AFFF-GW",  "LL", "BL", "WWTP", "PP", "PG"], loc="upper right")
#plt.title("Class breakdown via RF vs. Original Class Label", fontsize = 16)
plt.savefig('SVC-MultiClass-RNKU-Normalized_NEW.png', dpi = 1500, bbox_inches='tight')
plt.show()

In [ ]:
#create bar plot
x = ['AFFF-GW', 'LL', 'BL', 'WWTP', 'PP', 'PG']

plt.figure(figsize=(10, 6)) 
plt.bar(x, y1, color ='#7ad151')
plt.bar(x, y2, bottom=y1, color='#22a884')
plt.bar(x, y3, bottom=y1+y2, color='#fde725')
plt.bar(x, y4, bottom=y1+y2+y3, color='#440154')
plt.bar(x, y5, bottom=y1+y2+y3+y4, color='#414487')
plt.bar(x, y6, bottom=y1+y2+y3+y4+y5, color='#2a788e')
plt.yticks(np.arange(0, 29, 2), fontsize = 12)
plt.xticks(fontsize = 12)
plt.xlabel("SVC Groups", fontsize = 14)
plt.ylabel("Frequency", fontsize = 14)
for i, (yi, yj, yk, yl, ym, yn) in enumerate(zip(y1, y2, y3, y4, y5, y6)):
    total_count = int(yi + yj + yk + yl + ym + yn)  # Convert to int to remove decimals
    plt.text(i, total_count + 0.5, str(total_count), ha="center")
plt.legend(["AFFF-GW",  "LL", "BL", "WWTP", "PP", "PG"], loc="upper right")
#plt.title("Class breakdown via RF vs. Original Class Label", fontsize = 16)
plt.savefig('SVC-MultiClass-RNKU-Normalized-Legend_NEW.png', dpi = 1500, bbox_inches='tight')
plt.show()

### Decision Tree

In [ ]:
#Running randomized CV to narrow grid for GridSearch CV
#Modified from here: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

# Citerion to measure quality of a plit
crit = ['gini', 'entropy']
# Number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Create the random grid
random_grid = {'criterion': crit,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
print(random_grid)

In [ ]:
# Split the data into training and held-out validation set
data_train, data_val, target_train, target_val = train_test_split(data_1, target_1, test_size=0.2, random_state=42)

# Use the random grid to search for best hyperparameters
# First create the base model to tune
model_rnd = DecisionTreeClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
dt_random = RandomizedSearchCV(estimator = model_rnd, param_distributions = random_grid, n_iter = 100, cv = 7, verbose=2, random_state=42, n_jobs = -1)

dt_random.fit(data_train, target_train)

# Print the best parameters found
print("Best Parameters:", dt_random.best_params_)


In [ ]:
# Evaluate the best model on the held-out validation set
dt_model = dt_random.best_estimator_
val_predictions = dt_model.predict(data_val)
val_balanced_accuracy = balanced_accuracy_score(target_val, val_predictions)

print("Validation Balanced Accuracy:", val_balanced_accuracy)

In [ ]:
# Split the data into training and held-out validation set
data_train, data_val, target_train, target_val = train_test_split(data_1, target_1, test_size=0.2, random_state=42)

#Defining model based on results from randomized CV
model = DecisionTreeClassifier()

#Parameters to explore
min_samples_split = [1, 2]
min_samples_leaf = [1, 2]
max_features = ['sqrt']
max_depth = [70, 80, 90]
criterion = ['entropy']

#Create a custom scorer for balanced accuracy
balanced_accuracy_scorer = make_scorer(balanced_accuracy_score)

#Defining grid and performing CV of training data
grid = dict(min_samples_split = min_samples_split, min_samples_leaf = min_samples_leaf, max_features = max_features, max_depth = max_depth, criterion=criterion)
cv = RepeatedStratifiedKFold(n_splits=7, n_repeats=100, random_state=42)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring=balanced_accuracy_scorer, error_score=0)
grid_result = grid_search.fit(data_train, target_train.ravel())

# Summarize results of training data
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Evaluate the best model on the held-out validation set
best_model = grid_result.best_estimator_
val_predictions = best_model.predict(data_val)
val_balanced_accuracy = balanced_accuracy_score(target_val, val_predictions)

print("Validation Balanced Accuracy:", val_balanced_accuracy)

In [ ]:
#Now running optimized model over the entire dataset
model = DecisionTreeClassifier(min_samples_split = 2, min_samples_leaf = 1, max_features = 'sqrt', max_depth = 70, criterion='entropy')
# fit the model
model.fit(data_1, target_1.ravel())
acc = model.score(data_1, target_1)

print("DT accuracy: " + str(round(acc, 3)))

In [ ]:
# Predict class labels for the data
predicted_labels = model.predict(data_1)

# Initialize lists to store data points, row numbers, and original target labels for each class
class_data = [[] for _ in range(6)]  # Assuming 6 classes
class_row_numbers = [[] for _ in range(6)]
class_original_labels = [[] for _ in range(6)]

# Organize samples into their respective classes
for row_number, (predicted_label, original_label) in enumerate(zip(predicted_labels, target_1)):
    class_data[int(predicted_label)].append(data_1[row_number])
    class_row_numbers[int(predicted_label)].append(row_number)
    class_original_labels[int(predicted_label)].append(original_label)

# Print information for each class
for class_idx in range(6):
    print(f"Class {class_idx}:")
    print(f"Number of data points: {len(class_data[class_idx])}")
    print(f"Row numbers: {class_row_numbers[class_idx]}")
    print(f"Original target labels: {class_original_labels[class_idx]}")
    print()

In [ ]:
# Mapping from class numbers to class types
class_mapping = {
    0.0: "AFFF-GW",
    1.0: "LL",
    2.0: "BL",
    3.0: "WWTP",
    4.0: "PP",
    5.0: "PG"
}

# Replace class numbers with class types in class_original_labels
class_original_labels = [
    [class_mapping.get(label, label) for label in class_labels]
    for class_labels in class_original_labels
]
print(class_original_labels)

In [ ]:
# Define class types in the same order as the mapping
class_types = ["AFFF-GW", "LL", "BL", "WWTP", "PP", "PG"]

# Initialize six NumPy arrays to count instances for each class type
y1 = np.zeros(len(class_original_labels))
y2 = np.zeros(len(class_original_labels))
y3 = np.zeros(len(class_original_labels))
y4 = np.zeros(len(class_original_labels))
y5 = np.zeros(len(class_original_labels))
y6 = np.zeros(len(class_original_labels))

# Iterate through class_original_labels to count instances for each class type
for i, labels in enumerate(class_original_labels):
    for j, class_type in enumerate(class_types):
        # Count the instances of the current class type in the current class
        count = labels.count(class_type)
        
        # Update the respective NumPy array (e.g., y1, y2, etc.)
        if j == 0:
            y1[i] = count
        elif j == 1:
            y2[i] = count
        elif j == 2:
            y3[i] = count
        elif j == 3:
            y4[i] = count
        elif j == 4:
            y5[i] = count
        elif j == 5:
            y6[i] = count
            
print(y1)
print(y2)
print(y3)
print(y4)
print(y5)
print(y6)

In [ ]:
#create bar plot
x = ['AFFF-GW', 'LL', 'BL', 'WWTP', 'PP', 'PG']

plt.figure(figsize=(10, 6)) 
plt.bar(x, y1, color ='#7ad151')
plt.bar(x, y2, bottom=y1, color='#22a884')
plt.bar(x, y3, bottom=y1+y2, color='#fde725')
plt.bar(x, y4, bottom=y1+y2+y3, color='#440154')
plt.bar(x, y5, bottom=y1+y2+y3+y4, color='#414487')
plt.bar(x, y6, bottom=y1+y2+y3+y4+y5, color='#2a788e')
plt.yticks(np.arange(0, 29, 2), fontsize = 12)
plt.xticks(fontsize = 12)
plt.xlabel("DT Groups", fontsize = 14)
plt.ylabel("Frequency", fontsize = 14)
for i, (yi, yj, yk, yl, ym, yn) in enumerate(zip(y1, y2, y3, y4, y5, y6)):
    total_count = int(yi + yj + yk + yl + ym + yn)  # Convert to int to remove decimals
    plt.text(i, total_count + 0.5, str(total_count), ha="center")
#plt.legend(["AFFF-GW",  "LL", "BL", "WWTP", "PP", "PG"], loc="upper right")
#plt.title("Class breakdown via RF vs. Original Class Label", fontsize = 16)
plt.savefig('DT-MultiClass-RNKU-Normalized_NEW.png', dpi = 1500, bbox_inches='tight')
plt.show()

In [ ]:
#create bar plot
x = ['AFFF-GW', 'LL', 'BL', 'WWTP', 'PP', 'PG']

plt.figure(figsize=(10, 6)) 
plt.bar(x, y1, color ='#7ad151')
plt.bar(x, y2, bottom=y1, color='#22a884')
plt.bar(x, y3, bottom=y1+y2, color='#fde725')
plt.bar(x, y4, bottom=y1+y2+y3, color='#440154')
plt.bar(x, y5, bottom=y1+y2+y3+y4, color='#414487')
plt.bar(x, y6, bottom=y1+y2+y3+y4+y5, color='#2a788e')
plt.yticks(np.arange(0, 29, 2), fontsize = 12)
plt.xticks(fontsize = 12)
plt.xlabel("DT Groups", fontsize = 14)
plt.ylabel("Frequency", fontsize = 14)
for i, (yi, yj, yk, yl, ym, yn) in enumerate(zip(y1, y2, y3, y4, y5, y6)):
    total_count = int(yi + yj + yk + yl + ym + yn)  # Convert to int to remove decimals
    plt.text(i, total_count + 0.5, str(total_count), ha="center")
plt.legend(["AFFF-GW",  "LL", "BL", "WWTP", "PP", "PG"], loc="upper right")
#plt.title("Class breakdown via RF vs. Original Class Label", fontsize = 16)
plt.savefig('DT-MultiClass-RNKU-Normalized-Legend_NEW.png', dpi = 1500, bbox_inches='tight')
plt.show()

### Extreme Gradient Boosting

In [ ]:
#Running randomized CV to narrow grid for GridSearch CV
#Modified from here: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
random_grid = {'gamma': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4, 200],
              'learning_rate': [0.01, 0.03, 0.06, 0.1, 0.15, 0.2, 0.25, 0.300000012, 0.4, 0.5, 0.6, 0.7],
              'max_depth': [5,6,7,8,9,10,11,12,13,14],
              'n_estimators': [50,65,80,100,115,130,150],
              'reg_alpha': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200],
              'reg_lambda': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200]}

print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
model_rnd = xgb.XGBClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = model_rnd, param_distributions = random_grid, n_iter = 100, cv = 7, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(data_1, target_1)
print(rf_random.best_params_)

In [ ]:
### Defining model based on results from randomized CV
model = xgb.XGBClassifier()

#Parameters to explore

reg_lambda = [0.1, 0.2, 0.3]
reg_alpha = [0.6, 0.8, 1.0]
n_estimators = [50, 100]
max_depth = [10, 11, 12]
learning_rate = [0.1]
gamma = [0, 0.2, 0.4]

#Create a custom scorer for balanced accuracy
balanced_accuracy_scorer = make_scorer(balanced_accuracy_score)

#Defining grid and performing CV of training data
grid = dict(reg_lambda=reg_lambda, reg_alpha=reg_alpha, n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, gamma=gamma)
cv = RepeatedStratifiedKFold(n_splits=7, n_repeats=100, random_state=42)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring=balanced_accuracy_scorer, error_score=0)
grid_result = grid_search.fit(data_1, target_1.ravel())

# summarize results of training data
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
#Now running optimized model over the entire dataset
model = xgb.XGBClassifier(gamma = 0, learning_rate = 0.55, max_depth = 4, n_estimators = 60, reg_alpha = 0.6, reg_lambda = 13.2)
# fit the model
model.fit(data_1, target_1.ravel())
acc = model.score(data_1, target_1)

print("XGBC accuracy: " + str(round(acc, 3)))

In [ ]:
# Predict class labels for the data
predicted_labels = model.predict(data_1)

# Initialize lists to store data points, row numbers, and original target labels for each class
class_data = [[] for _ in range(6)]  # Assuming 6 classes
class_row_numbers = [[] for _ in range(6)]
class_original_labels = [[] for _ in range(6)]

# Organize samples into their respective classes
for row_number, (predicted_label, original_label) in enumerate(zip(predicted_labels, target_1)):
    class_data[int(predicted_label)].append(data_1[row_number])
    class_row_numbers[int(predicted_label)].append(row_number)
    class_original_labels[int(predicted_label)].append(original_label)

# Print information for each class
for class_idx in range(6):
    print(f"Class {class_idx}:")
    print(f"Number of data points: {len(class_data[class_idx])}")
    print(f"Row numbers: {class_row_numbers[class_idx]}")
    print(f"Original target labels: {class_original_labels[class_idx]}")
    print()

In [ ]:
# Mapping from class numbers to class types
class_mapping = {
    0.0: "AFFF-GW",
    1.0: "LL",
    2.0: "BL",
    3.0: "WWTP",
    4.0: "PP",
    5.0: "PG"
}

# Replace class numbers with class types in class_original_labels
class_original_labels = [
    [class_mapping.get(label, label) for label in class_labels]
    for class_labels in class_original_labels
]
print(class_original_labels)

In [ ]:
# Define class types in the same order as the mapping
class_types = ["AFFF-GW", "LL", "BL", "WWTP", "PP", "PG"]

# Initialize six NumPy arrays to count instances for each class type
y1 = np.zeros(len(class_original_labels))
y2 = np.zeros(len(class_original_labels))
y3 = np.zeros(len(class_original_labels))
y4 = np.zeros(len(class_original_labels))
y5 = np.zeros(len(class_original_labels))
y6 = np.zeros(len(class_original_labels))

# Iterate through class_original_labels to count instances for each class type
for i, labels in enumerate(class_original_labels):
    for j, class_type in enumerate(class_types):
        # Count the instances of the current class type in the current class
        count = labels.count(class_type)
        
        # Update the respective NumPy array (e.g., y1, y2, etc.)
        if j == 0:
            y1[i] = count
        elif j == 1:
            y2[i] = count
        elif j == 2:
            y3[i] = count
        elif j == 3:
            y4[i] = count
        elif j == 4:
            y5[i] = count
        elif j == 5:
            y6[i] = count
            
print(y1)
print(y2)
print(y3)
print(y4)
print(y5)
print(y6)

In [ ]:
#create bar plot
x = ['AFFF-GW', 'LL', 'BL', 'WWTP', 'PP', 'PG']

plt.figure(figsize=(10, 6)) 
plt.bar(x, y1, color ='#7ad151')
plt.bar(x, y2, bottom=y1, color='#22a884')
plt.bar(x, y3, bottom=y1+y2, color='#fde725')
plt.bar(x, y4, bottom=y1+y2+y3, color='#440154')
plt.bar(x, y5, bottom=y1+y2+y3+y4, color='#414487')
plt.bar(x, y6, bottom=y1+y2+y3+y4+y5, color='#2a788e')
plt.yticks(np.arange(0, 29, 2), fontsize = 12)
plt.xticks(fontsize = 12)
plt.xlabel("XGBoost Groups", fontsize = 14)
plt.ylabel("Frequency", fontsize = 14)
for i, (yi, yj, yk, yl, ym, yn) in enumerate(zip(y1, y2, y3, y4, y5, y6)):
    total_count = int(yi + yj + yk + yl + ym + yn)  # Convert to int to remove decimals
    plt.text(i, total_count + 0.5, str(total_count), ha="center")
#plt.legend(["AFFF-GW",  "LL", "BL", "WWTP", "PP", "PG"], loc="upper right")
#plt.title("Class breakdown via RF vs. Original Class Label", fontsize = 16)
plt.savefig('XGBoost-MultiClass-RNKU-Normalized_NEW.png', dpi = 1500, bbox_inches='tight')
plt.show()

In [ ]:
#create bar plot
x = ['AFFF-GW', 'LL', 'BL', 'WWTP', 'PP', 'PG']

plt.figure(figsize=(10, 6)) 
plt.bar(x, y1, color ='#7ad151')
plt.bar(x, y2, bottom=y1, color='#22a884')
plt.bar(x, y3, bottom=y1+y2, color='#fde725')
plt.bar(x, y4, bottom=y1+y2+y3, color='#440154')
plt.bar(x, y5, bottom=y1+y2+y3+y4, color='#414487')
plt.bar(x, y6, bottom=y1+y2+y3+y4+y5, color='#2a788e')
plt.yticks(np.arange(0, 29, 2), fontsize = 12)
plt.xticks(fontsize = 12)
plt.xlabel("XGBoost Groups", fontsize = 14)
plt.ylabel("Frequency", fontsize = 14)
for i, (yi, yj, yk, yl, ym, yn) in enumerate(zip(y1, y2, y3, y4, y5, y6)):
    total_count = int(yi + yj + yk + yl + ym + yn)  # Convert to int to remove decimals
    plt.text(i, total_count + 0.5, str(total_count), ha="center")
plt.legend(["AFFF-GW",  "LL", "BL", "WWTP", "PP", "PG"], loc="upper right")
#plt.title("Class breakdown via RF vs. Original Class Label", fontsize = 16)
plt.savefig('XGBoost-MultiClass-RNKU-Normalized-Legend_NEW.png', dpi = 1500, bbox_inches='tight')
plt.show()